In [6]:
!pip install mesa
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd
import random

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [68]:
def obtener_calle(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    #Aqui se asignas los colores de las celdas del grid para la visualizacion de la simulacion =D muito bonito, joga bonito
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content,car):  
                grid[x][y] = 30  #color azul para el auto
            elif isinstance(content,Banqueta): 
                grid[x][y] = 100 #color blanco de la calle (bordes) 
            elif isinstance (content, sensores):
                grid[x][y] = 10 #color morado de los sensores
            elif isinstance(content, Semaforo):
                if content.estado_luz == 0: 
                    grid[x][y] = 60 #color verde de semaforo
                elif content.estado_luz == 1:
                    grid[x][y] = 90 #color rojo de semaforo 
               
            else: 
                grid[x][y] = 0  #color default del fondo 
    return grid


class Semaforo(Agent): #modelacion de los carritos muito bonitos
    
    def __init__(self, unique_id, Turno,color, model):
        super().__init__(unique_id,model)
        self.Turno = Turno ## indica si es el turno de que el carro avance o no :3
        self.estado_luz = color # 0 es pasa, 1 para 

    def step(self): 
        if self.Turno == self.model.roadTurn: 
            self.estado_luz = 0 
        else: 
            self.estado_luz = 1

class sensores(Agent): #modelacion de sensores que detectas vehiculos 
    def __init__(self,unique_id, number, model):
        super().__init__(unique_id,model)
        self.hasCar = False
        self.counter = 0
        self.myLight = number  

      
    def changeTurn(self):
        if self.model.roadTurn == 0:
            self.model.roadTurn = 1
        elif self.model.roadTurn == 1:
            self.model.roadTurn = 0          

    def step(self):
        thisCell = self.model.grid.get_cell_list_contents([self.pos])
        thisLightCheck = len([obj for obj in thisCell if isinstance(obj,car)])
        if thisLightCheck > 0:
            self.counter = 0
            self.hasCar = True
        else: 
            self.hasCar = False
            self.counter += 1
          
        if self.counter == 7:
            if self.myLight == 0:
                thisOtherCell = self.model.grid.get_cell_list_contents([(8,9)])
                light_object = [obj for obj in thisOtherCell if isinstance(obj, Semaforo)]
                light_object[0].estado_luz = 1
                self.changeTurn()
            if self.myLight == 1:
                thisOtherCell = self.model.grid.get_cell_list_contents([(5,7)])
                light_object = [obj for obj in thisOtherCell if isinstance(obj, Semaforo)]
                light_object[0].estado_luz = 1
                self.changeTurn()
            self.counter = 0

class Banqueta(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id,model)

class car(Agent):
    def __init__(self, unique_id, direction ,pos, model):
        super().__init__(unique_id, model)
        self.pos = pos
        self.direction = direction # 0 izquierda, 1 derecha, 2 arriba, 3 abajo 
        self.stop = False
        
    def getNextMove(self):
        if self.pos[1] == model.width -1 and self.direction == 1:
            nextPos = (9,0)
            return nextPos
        
        nextPos = (0,0)
        if self.direction == 0: #avanza a la direccion correspondienTec
            nextPos = (self.pos[0], self.pos[1] - 1)
        elif self.direction == 1:
            nextPos = (self.pos[0], self.pos[1] + 1)
        elif self.direction == 2:
            nextPos = (self.pos[0] -1 , self.pos[1])
        elif self.direction == 3:
            nextPos = (self.pos[0] + 1, self.pos[1])
        return nextPos
    def drive(self):
        thisCell = self.model.grid.get_cell_list_contents([self.pos])
        isLC = len([obj for obj in thisCell if isinstance(obj,sensores)])
        if isLC > 0:

          #checamos hacia donde se dirigen
            if self.direction == 1:
                thisOtherCell = self.model.grid.get_cell_list_contents([(8,9)])
                light_object = [obj for obj in thisOtherCell if isinstance(obj, Semaforo)]
            if self.direction == 2:
                thisOtherCell = self.model.grid.get_cell_list_contents([(5,7)])
                light_object = [obj for obj in thisOtherCell if isinstance(obj, Semaforo)]

          #ahora checamos el semaforo
            if light_object[0].estado_luz == 1:
                self.stop = True
            else: self.stop = False

            if not self.stop:
                nextPos = self.getNextMove()
                thisCell = self.model.grid.get_cell_list_contents([nextPos])
                howManyCars = len([obj for obj in thisCell if isinstance(obj,car)])
                if howManyCars == 0:
                    self.model.grid.move_agent(self,nextPos)
    
    def step(self):
        self.drive()


class Interseccion_calle(Model):
     def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        self.roadTurn = 0 # 0 a 1 en este caso
        #inicializamos los objetos
        counterX = 0
        y_v = 6

        for h in range (width):
            for k in range(height):
                if h == 5 and (k < 6 or k > 9):
                    varB = Banqueta(h+700,self)
                    self.grid.place_agent(varB,(k,h))

                if h == 9 and (k < 6 or k > 9):
                    varC = Banqueta(h+600,self)
                    self.grid.place_agent(varC,(k,h))
      
        for i in range (12):
            if i == 6:
                counterX = 0
                y_v = 10
            z = Banqueta(i,self)
            self.schedule.add(z)
            self.grid.place_agent(z,(y_v,counterX))
            counterX += 1
         
        counterX = 9
        y_v = 6

        for j in range (12):
            if j == 6:
                counterX = 9
                y_v =  10
            var = Banqueta(j+100, self)
            self.schedule.add(var)
            self.grid.place_agent(var,(y_v, counterX))
            counterX += 1

        for l in range(1):
            x = random.randint(2,((self.width * self.height) - 1))
            y = random.randint(2,((self.width * self.height) - 1))
            b = car((l + 200, l + 2001), 1,(y,x),self)
            self.schedule.add(b)
            self.grid.place_agent(b,(9,0))
            b2 = car((l + 20, l + 201),2,(y,x),self)
            self.schedule.add(b2)
            self.grid.place_agent(b2,(14,8))
        
        #crear 2 lightChecks en sus posiciones respectivas
        lc_1 = sensores(900, 0, self)
        self.schedule.add(lc_1)
        self.grid.place_agent(lc_1,(9,5))

        lc_arriba = sensores(901, 1, self)
        self.schedule.add(lc_arriba)
        self.grid.place_agent(lc_arriba,(10,8))

        #crear 2 semaforos en sus posiciones respectivas
        semaforo_izquierda = Semaforo(510,0,2,self)
        self.schedule.add(semaforo_izquierda)
        self.grid.place_agent(semaforo_izquierda,(8,9))

        semaforo_arriba = Semaforo(511,1,2,self)
        self.schedule.add(semaforo_arriba)
        self.grid.place_agent(semaforo_arriba,(5,7))
        


        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(model_reporters={"Grid": obtener_calle})
   

     def step(self) :
        self.datacollector.collect(self)       
        self.schedule.step()

M = 15
N = 15

MAX_TIME = 0.03

start_time = time.time()
model = Interseccion_calle(M, N)

while((time.time() - start_time) < MAX_TIME):
    model.step()

all_grid = model.datacollector.get_model_vars_dataframe()
all_grid.to_string()


'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [65]:
%%capture
fig, axs = plt.subplots(figsize=(5,5))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap= 'nipy_spectral')

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate,frames = len(all_grid)) 

In [44]:
anim